In [ ]:
import os
import google.generativeai as genai

class Product:
    def __init__(self, name, base_price):
        self.name = name
        self.base_price = base_price

class NegotiationChatbot:
    def __init__(self, product, min_discount, max_discount):
        self.product = product
        self.min_discount = min_discount
        self.max_discount = max_discount
        self.current_price = product.base_price
        self.negotiation_rounds = 0
        self.max_rounds = 7
        self.model = genai.GenerativeModel('gemini-pro')
        self.chat = self.model.start_chat(history=[])

    def get_ai_response(self, user_message):
        system_prompt = f"""
        You are a salesperson negotiating the price of a {self.product.name}.
        Current price: ${self.current_price}
        Base price: ${self.product.base_price}
        Negotiation round: {self.negotiation_rounds + 1} of {self.max_rounds}

        Respond as a determined salesperson focused on securing a deal with minimal discount from the base price. 
        Emphasize the product's value and fairness of the base price, and make only small concessions. 
        Encourage the customer to close the deal soon.
        Keep coversation short and concise.
        Address the customer directly, using "you" instead of "the customer."
        """
        response = self.chat.send_message(f"{system_prompt}\n\nUser: {user_message}")
        return response.text

    def calculate_discount(self, user_offer):
        if user_offer < self.product.base_price * (1 - self.max_discount):
            return self.max_discount
        elif user_offer > self.current_price:
            return 0  # No discount if the offer is higher than the current price
        else:
            discount = (self.current_price - user_offer) / self.product.base_price
            return max(min(discount, self.min_discount + 0.02), self.min_discount)  # Only small concessions

    def negotiate(self, user_message):
        self.negotiation_rounds += 1

        if self.negotiation_rounds > self.max_rounds:
            return f"I apologize, but we've reached our maximum number of negotiation rounds. My final offer stands at ${self.current_price}. Would you like to proceed with this price?"

        if 'accept' in user_message.lower():
            return f"Deal closed at ${self.current_price}! Thank you for your business."
        
        if 'reject' in user_message.lower():
            return "I understand. If you change your mind or have any further questions, feel free to ask."

        try:
            user_offer = float(user_message)
            if user_offer < self.product.base_price * (1 - self.max_discount):
                return self.get_ai_response(f"You've offered ${user_offer}, which is below our acceptable range. We cannot go that low given the product's value.")

            discount = self.calculate_discount(user_offer)
            new_price = round(self.product.base_price * (1 - discount), 2)

            # Ensure the price decreases minimally with each negotiation
            if new_price < self.current_price:
                self.current_price = new_price

            response = self.get_ai_response(f"You've offered ${user_offer}. Based on your offer, we can slightly reduce the price to ${self.current_price}. This is a great deal considering the product's value. What do you think?")

            # Check if the response contains a deal or rejection
            if "deal" in response.lower() or "accept" in user_message.lower():
                return f"Deal closed at ${self.current_price}! Thank you for your business."
            elif "reject" in response.lower():
                return "I understand. If you change your mind or have any further questions, feel free to ask."
            else:
                return response

        except ValueError:
            return "Please enter a valid number for your desired price or type 'accept' or 'reject'."

if __name__ == "__main__":
    # Load your API key from an environment variable
    genai.configure(api_key="Your API Key")

    # Create a product instance
    product = Product("Laptop", 2000)

    # Create a negotiation chatbot instance
    chatbot = NegotiationChatbot(product, 0.01, 0.05)

    # Start the negotiation
    print("Welcome to the negotiation for the Laptop. The base price is 2000 $.")
    print("Feel free to propose a price or type 'accept' or 'reject'.")
    while True:
        user_input = input("Enter your desired price or 'accept'/'reject': ")
        response = chatbot.negotiate(user_input)
        print(response)

        # Break the loop if a deal is closed or rejected
        if "deal closed" in response.lower() or "reject" in response.lower():
            break
